# Extracting turn around times 

In [3]:
%matplotlib inline
import pandas as pd
import os, glob
import matplotlib.backends.backend_pdf
import matplotlib.pyplot as plt
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import scipy
from scipy.stats import zscore
from scipy.signal import savgol_filter
import matplotlib.gridspec as gridspec

In [4]:
path = r'C:\Users\jessi\Documents\Touch Data\Turn around times'

Set up participant number

In [151]:
participantNumber = ['P01', 'P02', 'P03','P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'P13', 'P14',
                    'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28',
                     'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35']

Set up block number

In [6]:
blockNumber = ['B1', 'B2', 'B3']

Set up fabric code 

In [7]:
fabricCode = ['A', 'B', 'C']

Set up episode number 

In [8]:
episodeNumber = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0]

Read in compilation table

In [9]:
df = pd.read_csv(r'C:\Users\jessi\Documents\Touch Data\Turn around times\CompilationTable.csv')

Remove irrelevent columns from dataframe

In [10]:
df = df.drop(
    ['File_Number', 'Reference', 'Protocol', 'Analysis', 'Date_Stamp', 'Time_Stamp', 'DataSaveBlock',
     'BlockStamp_s', 'BrickInBlock', 'Time_s', 'Adjustment_s', 'RelativeTime_s', 'Trigger_V', 'SensorA_Abs_V',
     'SensorB_Abs_V', 'SensorA_Net_V', 'SensorB_Net_V', 'Study', 'ForceAtXminus47p5mm_g', 'ForceAtXplus47p5mm_g'],
    axis=1)


Filter Data

In [11]:
dfFiltered = df.loc[~(df['Episode_Number'] ==0)]
dfFiltered = dfFiltered[dfFiltered['Position_mm'] > -100]
dfFiltered = dfFiltered[dfFiltered['Position_mm'] < 100]
dfFiltered = dfFiltered[dfFiltered['EpisodeRelativeTime_s'] >= 0]
dfFiltered= dfFiltered[dfFiltered['EpisodeRelativeTime_s'] < 7.5]
dfFiltered = dfFiltered[dfFiltered['TotalLoad_g'] > 1]
#dfFiltered = dfFiltered.reset_index()
#dfFiltered

Create empty dataframes

In [257]:
dfTurnAroundposition = pd.DataFrame(columns=['Participant', 'Block', 'Fabric', 'Episode Number', 
                                             'touchDown','sweepStart','leftTurn1', 'rightTurn', 
                                             'leftTurn2', 'sweepEnd', 'liftOff', 
                                             'touchDownPosition', 'sweepStartPosition','leftTurn1Position',
                                             'rightTurnPosition', 'leftTurn2Position', 'sweepEndPosition',
                                             'liftOffPosition', 'epochEnd1', 'epochStart2', 'epochEnd2', 'epochStart3',
                                             'sweep1TotalTime', 'sweep2TotalTime', 'sweep3TotalTime', 'sweep4TotalTime'])

In [258]:
dfTotalLoad = pd.DataFrame(columns=['Participant', 'Block', 'Fabric', 'Episode Number', 'meanLoad', 'medianLoad'])

In [259]:
dfOriginalDataSmooth = pd.DataFrame()

Loop to calculate turn around times

In [260]:
for p in participantNumber:
    dfP = dfFiltered[dfFiltered['Participant'] == p]  # when value in 'Participant' column is equal to the participant value in loop
    for b in blockNumber:
        dfB = dfP[dfP['Study_Block'] == b]  # when value in 'Study_Block' column is equal to theStudy_block value in loop
        for f in fabricCode:
            dfF = dfB[dfB['Fabric'] == f]  # when value in 'Fabric' column is equal to the fabric value in loop
            dfTurnAroundTemp = pd.DataFrame(columns=['Participant', 'Block', 'Fabric', 'Episode Number', 'touchDown','leftTurn1', 'rightTurn', 'leftTurn2', 'liftOff'])
            for e in episodeNumber:
                dfE = dfF[dfF['Episode_Number'] == e]  # when value in 'Episode number' column is equal to the episode value in loop
                dfOriginalData = dfE.reset_index() #Reset the index and add the old index as a column
                
                dfShape = len(dfOriginalData.index) #calculate the amount of rows in the dataframe
                if (dfShape <= 10): #skip over dataframes that have less than 10 rows
                    dfTurnAroundposition = dfTurnAroundposition.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'touchDown': np.nan, 'sweepStart': np.nan, 'leftTurn1': np.nan, 'rightTurn': np.nan, 'leftTurn2': np.nan, 'sweepEnd': np.nan,'liftOff': np.nan, 'touchDownPosition': np.nan, 'sweepStartPosition': np.nan,'leftTurn1Position': np.nan, 'rightTurnPosition': np.nan, 'leftTurn2Position': np.nan, 'sweepEndPosition': np.nan, 'liftOffPosition': np.nan, 'epochEnd1': np.nan, 'epochStart2': np.nan, 'epochEnd2': np.nan, 'epochStart3': np.nan, 'sweep1TotalTime': np.nan, 'sweep2TotalTime': np.nan, 'sweep3TotalTime': np.nan, 'sweep4TotalTime': np.nan},ignore_index=True)
                    dfTotalLoad = dfTotalLoad.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'meanLoad': np.nan, 'medianLoad': np.nan},ignore_index=True)
                    continue
                
                dfPosition = dfOriginalData['Position_mm'] #create a variable that is just the positon column 
                smooth = dfPosition.rolling(window=3, win_type='gaussian', center=True).mean(std=0.5) #smooth the data using a gaussian filter    
                smooth.iloc[0] = dfPosition.iloc[0]
                smooth.iloc[-1] = dfPosition.iloc[-1]
                dfOriginalData['PositionSmooth'] = smooth # append the smoooth data to the orignical dataframe under the column neame 'PositionSmooth'
                dfOriginalDataSmooth = dfOriginalDataSmooth.append(dfOriginalData) #Append the data to the empty dataframe outside the loop
                
                dfMinIndex = dfOriginalData.at[0, 'index'] #find the first value in  the 'index' column
                dfSecondIndex = dfOriginalData.at[1,'index'] #find the second value in the 'index' column 
                dfIndexDifferenceCalculation = dfSecondIndex - dfMinIndex #Calculate the differnece between the second and first index value
               

                if (dfIndexDifferenceCalculation > 1): #if the index calculation is lower than 1 then drop the fist row
                    touchDownTimeFiltered = dfOriginalData.drop(df.index[0])
                    
                else:
                    touchDownTimeFiltered = dfOriginalData
                    
                #find the touch down and lift off times 
                touchDownTimeFiltered = touchDownTimeFiltered.reset_index() # Rest the index because we just droped a row, so we rest to make sure the first row is now 0 
                touchDownStartTime = touchDownTimeFiltered['EpisodeRelativeTime_s'].iloc[0] # Get the first value in the episode relative time column
                touchDownEndTime = touchDownTimeFiltered['EpisodeRelativeTime_s'].iloc[-1] # Get the last value in the episode realtive time column
                TouchDownPostion = touchDownTimeFiltered['PositionSmooth'].iloc[0]
                LiftOffPosition = touchDownTimeFiltered['PositionSmooth'].iloc[-1]
                
                if touchDownStartTime >= 1 or touchDownStartTime <= 0.2: # reject trials where the touch down time is greater than one second or less than 0.2 seconds
                    dfTurnAroundposition = dfTurnAroundposition.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'touchDown': np.nan, 'sweepStart': np.nan, 'leftTurn1': np.nan, 'rightTurn': np.nan, 'leftTurn2': np.nan, 'sweepEnd': np.nan,'liftOff': np.nan, 'touchDownPosition': np.nan, 'sweepStartPosition': np.nan,'leftTurn1Position': np.nan, 'rightTurnPosition': np.nan, 'leftTurn2Position': np.nan, 'sweepEndPosition': np.nan, 'liftOffPosition': np.nan, 'epochEnd1': np.nan, 'epochStart2': np.nan, 'epochEnd2': np.nan, 'epochStart3': np.nan, 'sweep1TotalTime': np.nan, 'sweep2TotalTime': np.nan, 'sweep3TotalTime': np.nan, 'sweep4TotalTime': np.nan},ignore_index=True)
                    dfTotalLoad = dfTotalLoad.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'meanLoad': np.nan, 'medianLoad': np.nan},ignore_index=True)
                    continue
                    
                if touchDownEndTime <= 4 or touchDownEndTime >= 7: # reject trials where the lift off time is less han 4 seconds or greater than 7 seconds
                    dfTurnAroundposition = dfTurnAroundposition.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'touchDown': np.nan, 'sweepStart': np.nan, 'leftTurn1': np.nan, 'rightTurn': np.nan, 'leftTurn2': np.nan, 'sweepEnd': np.nan,'liftOff': np.nan, 'touchDownPosition': np.nan, 'sweepStartPosition': np.nan,'leftTurn1Position': np.nan, 'rightTurnPosition': np.nan, 'leftTurn2Position': np.nan, 'sweepEndPosition': np.nan, 'liftOffPosition': np.nan, 'epochEnd1': np.nan, 'epochStart2': np.nan, 'epochEnd2': np.nan, 'epochStart3': np.nan, 'sweep1TotalTime': np.nan, 'sweep2TotalTime': np.nan, 'sweep3TotalTime': np.nan, 'sweep4TotalTime': np.nan},ignore_index=True)
                    dfTotalLoad = dfTotalLoad.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'meanLoad': np.nan, 'medianLoad': np.nan},ignore_index=True)
                    continue
                    
                #Calculate epoch time periods for sweep start and sweep end
                totalTouchTime = touchDownEndTime - touchDownStartTime #calculate the touch period 
                startEndEpoch = touchDownEndTime/4
                
                #Calculate epoch start and end times
                epoch1End = touchDownStartTime + startEndEpoch
                epoch2Start = touchDownEndTime - startEndEpoch 
                
                #Give the dataframe a new name so it makes sense for the calcualtions
                turnAroundCalculations = touchDownTimeFiltered
               
                #epoch the data into 2 parts
                turnAroundCalculationsStart = turnAroundCalculations[(turnAroundCalculations['EpisodeRelativeTime_s'] >= touchDownStartTime) & (turnAroundCalculations['EpisodeRelativeTime_s'] <= epoch1End)]
                turnAroundCalculationsEnd = turnAroundCalculations[(turnAroundCalculations['EpisodeRelativeTime_s'] >= epoch2Start) & (turnAroundCalculations['EpisodeRelativeTime_s'] <= touchDownEndTime)]
                
                #Find the sweep start epoch
                sweepStart = turnAroundCalculationsStart[['PositionSmooth']].idxmin()
                sweepStartTime = turnAroundCalculationsStart.loc[sweepStart,['EpisodeRelativeTime_s']]
                sweepStartPosition = turnAroundCalculationsStart.loc[sweepStart,['PositionSmooth']]
                sweepStartTime = sweepStartTime.iat[0,0]
                sweepStartPosition = sweepStartPosition.iat[0,0]
                
                #Find the sweep end epoch
                sweepEnd = turnAroundCalculationsEnd[['PositionSmooth']].idxmin()
                sweepEndTime = turnAroundCalculationsEnd.loc[sweepEnd,['EpisodeRelativeTime_s']]
                sweepEndPosition = turnAroundCalculationsEnd.loc[sweepEnd,['PositionSmooth']]
                sweepEndTime = sweepEndTime.iat[0,0]
                sweepEndPosition = sweepEndPosition.iat[0,0]
                
                #Calculate total load mean and median from the start sweep to the end sweep
                toalLoadCalcuations = turnAroundCalculations[(turnAroundCalculations['EpisodeRelativeTime_s'] >= sweepStartTime) & (turnAroundCalculations['EpisodeRelativeTime_s'] <= sweepEndTime)]
                meanLoad = toalLoadCalcuations['TotalLoad_g'].mean()
                medianLoad = toalLoadCalcuations['TotalLoad_g'].median()
                
                #Calculate epoch time periods
                totalTouchTime = sweepEndTime - sweepStartTime #calculate the touch period 
                touchMiddlePoint = totalTouchTime/2 #calculate the middle point of the touch period
                thirdofTouchTimeMiddle = totalTouchTime/6 #calculate what one sixth of the touch time is
                plusMinusWindowMiddle = totalTouchTime/12 #Calculate an overlap for the window
                
                 #Calculate epoch start and end times
                epochStart2 = touchMiddlePoint- thirdofTouchTimeMiddle - plusMinusWindowMiddle
                epochEnd2 = touchMiddlePoint+ thirdofTouchTimeMiddle + plusMinusWindowMiddle
                epochLenghth = epochEnd2- epochStart2
                epochEnd1 = sweepStartTime + epochLenghth
                epochStart3 = sweepEndTime - epochLenghth
                
                #epoch the data into 3 parts
                turnAroundCalculations2 = turnAroundCalculations[(turnAroundCalculations['EpisodeRelativeTime_s'] >= sweepStartTime) & (turnAroundCalculations['EpisodeRelativeTime_s'] <= epochEnd1)]
                turnAroundCalculations4 = turnAroundCalculations[(turnAroundCalculations['EpisodeRelativeTime_s'] >= epochStart2) & (turnAroundCalculations['EpisodeRelativeTime_s'] <= epochEnd2)]
                turnAroundCalculations6 = turnAroundCalculations[(turnAroundCalculations['EpisodeRelativeTime_s'] >= epochStart3) & (turnAroundCalculations['EpisodeRelativeTime_s'] <= sweepEndTime)]

                 #find the index position for the turn around times
                LeftTurnAround = turnAroundCalculations2[['PositionSmooth']].idxmax()
                RightTurnARound = turnAroundCalculations4[['PositionSmooth']].idxmin()
                LeftTurnAround2 = turnAroundCalculations6[['PositionSmooth']].idxmax()
                
                #Use the index positions to find the corresponding episode time   
                leftTurnAroundTime = turnAroundCalculations2.loc[LeftTurnAround,['EpisodeRelativeTime_s']]
                rightTurnAroundTime = turnAroundCalculations4.loc[RightTurnARound,['EpisodeRelativeTime_s']]
                leftTurnAroundTime2 = turnAroundCalculations6.loc[LeftTurnAround2,['EpisodeRelativeTime_s']]
                
                #Use the index positions to find the corresponding position value 
                LeftTurnAround = turnAroundCalculations2.loc[LeftTurnAround,['PositionSmooth']]
                RightTurnARound = turnAroundCalculations4.loc[RightTurnARound,['PositionSmooth']]
                LeftTurnAround2 = turnAroundCalculations6.loc[LeftTurnAround2,['PositionSmooth']]
                
                #get the episode relative time as a vector
                leftTurnAroundTime = leftTurnAroundTime.iat[0,0]
                rightTurnAroundTime = rightTurnAroundTime.iat[0,0]
                leftTurnAroundTime2 = leftTurnAroundTime2.iat[0,0]
                
                #get the position as a vector
                LeftTurnAround = LeftTurnAround.iat[0,0]
                RightTurnARound = RightTurnARound.iat[0,0]
                LeftTurnAround2 = LeftTurnAround2.iat[0,0]
                
                #Calculate total sweep times
                sweep1 = leftTurnAroundTime - sweepStartTime
                sweep2 = rightTurnAroundTime - leftTurnAroundTime
                sweep3 = leftTurnAroundTime2 - rightTurnAroundTime
                sweep4 = sweepEndTime - leftTurnAroundTime2
                
                if sweep1 > epochLenghth or sweep2 > epochLenghth or sweep3 > epochLenghth or sweep4 > epochLenghth:
                    dfTurnAroundposition = dfTurnAroundposition.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'touchDown': np.nan, 'sweepStart': np.nan, 'leftTurn1': np.nan, 'rightTurn': np.nan, 'leftTurn2': np.nan, 'sweepEnd': np.nan,'liftOff': np.nan, 'touchDownPosition': np.nan, 'sweepStartPosition': np.nan,'leftTurn1Position': np.nan, 'rightTurnPosition': np.nan, 'leftTurn2Position': np.nan, 'sweepEndPosition': np.nan, 'liftOffPosition': np.nan, 'epochEnd1': np.nan, 'epochStart2': np.nan, 'epochEnd2': np.nan, 'epochStart3': np.nan, 'sweep1TotalTime': np.nan, 'sweep2TotalTime': np.nan, 'sweep3TotalTime': np.nan, 'sweep4TotalTime': np.nan},ignore_index=True)
                    dfTotalLoad = dfTotalLoad.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'meanLoad': np.nan, 'medianLoad': np.nan},ignore_index=True)
                    continue
                
                if sweep1 <= 0 or sweep2 <= 0 or sweep3 <= 0 or sweep4 <= 0:
                    dfTurnAroundposition = dfTurnAroundposition.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'touchDown': np.nan, 'sweepStart': np.nan, 'leftTurn1': np.nan, 'rightTurn': np.nan, 'leftTurn2': np.nan, 'sweepEnd': np.nan,'liftOff': np.nan, 'touchDownPosition': np.nan, 'sweepStartPosition': np.nan,'leftTurn1Position': np.nan, 'rightTurnPosition': np.nan, 'leftTurn2Position': np.nan, 'sweepEndPosition': np.nan, 'liftOffPosition': np.nan, 'epochEnd1': np.nan, 'epochStart2': np.nan, 'epochEnd2': np.nan, 'epochStart3': np.nan, 'sweep1TotalTime': np.nan, 'sweep2TotalTime': np.nan, 'sweep3TotalTime': np.nan, 'sweep4TotalTime': np.nan},ignore_index=True)
                    dfTotalLoad = dfTotalLoad.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'meanLoad': np.nan, 'medianLoad': np.nan},ignore_index=True)
                    continue
                
                
               
                #Append to empty data frame
                dfTurnAroundposition = dfTurnAroundposition.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 
                                                                    'touchDown': touchDownStartTime, 'sweepStart': sweepStartTime, 'leftTurn1': leftTurnAroundTime, 
                                                                    'rightTurn': rightTurnAroundTime, 'leftTurn2': leftTurnAroundTime2, 'sweepEnd': sweepEndTime,
                                                                    'liftOff': touchDownEndTime, 'touchDownPosition': TouchDownPostion, 'sweepStartPosition': sweepStartPosition,
                                                                    'leftTurn1Position': LeftTurnAround, 'rightTurnPosition': RightTurnARound, 
                                                                    'leftTurn2Position': LeftTurnAround2, 'sweepEndPosition': sweepEndPosition, 'liftOffPosition': LiftOffPosition, 
                                                                    'epochEnd1': epochEnd1, 'epochStart2': epochStart2, 'epochEnd2': epochEnd2, 
                                                                    'epochStart3': epochStart3, 'sweep1TotalTime': sweep1, 'sweep2TotalTime': sweep2, 
                                                                    'sweep3TotalTime': sweep3, 'sweep4TotalTime': sweep4},ignore_index=True)

                dfTotalLoad = dfTotalLoad.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'meanLoad': meanLoad, 'medianLoad': medianLoad},ignore_index=True)
                    

Remove sweeps based on SD 

In [261]:
dfTurnAroundFiltered = pd.DataFrame()

In [262]:
dfTotalLoadFiltered = pd.DataFrame()

In [264]:
for f in fabricCode:
    dfTurnAroundpositionSD = dfTurnAroundposition[dfTurnAroundposition['Fabric'] == f]  # when value in 'Participant' column is equal to the participant value in loop
    dfTotalLoadSD = dfTotalLoad[dfTotalLoad['Fabric'] == f]
    metadataTurnAround = dfTurnAroundpositionSD.iloc[:, 0:4]
    
    #Create z scores for the sweep data
    dfTurnAroundpositionSD = dfTurnAroundpositionSD.assign(sweep1Time_zscore = '', sweep2Time_zscore = '', sweep3Time_zscore = '', sweep4Time_zscore = '') #create a new empty column in data fram dfE called 'TotalLoad_g_zscore'
    dfTurnAroundpositionSD['sweep1Time_zscore'] = (dfTurnAroundpositionSD.sweep1TotalTime - dfTurnAroundpositionSD.sweep1TotalTime.mean())/dfTurnAroundpositionSD.sweep1TotalTime.std(ddof=0) # create zscore for 'TotalLoad_g' and put values in a new column of dfE named 'TotalLoad_g_zscore'
    dfTurnAroundpositionSD['sweep2Time_zscore'] = (dfTurnAroundpositionSD.sweep2TotalTime - dfTurnAroundpositionSD.sweep2TotalTime.mean())/dfTurnAroundpositionSD.sweep2TotalTime.std(ddof=0)
    dfTurnAroundpositionSD['sweep3Time_zscore'] = (dfTurnAroundpositionSD.sweep3TotalTime - dfTurnAroundpositionSD.sweep3TotalTime.mean())/dfTurnAroundpositionSD.sweep3TotalTime.std(ddof=0)
    dfTurnAroundpositionSD['sweep4Time_zscore'] = (dfTurnAroundpositionSD.sweep4TotalTime - dfTurnAroundpositionSD.sweep4TotalTime.mean())/dfTurnAroundpositionSD.sweep4TotalTime.std(ddof=0)
    
    #Create z score for the load data
    dfTotalLoadSD = dfTotalLoadSD.assign(mean_zscore = '', median_zscore = '')
    dfTotalLoadSD['mean_zscore'] = (dfTotalLoadSD.meanLoad - dfTotalLoadSD.meanLoad.mean())/dfTotalLoadSD.meanLoad.std(ddof=0) # create zscore for 'TotalLoad_g' and put values in a new column of dfE named 'TotalLoad_g_zscore'
    dfTotalLoadSD['median_zscore'] = (dfTotalLoadSD.medianLoad - dfTotalLoadSD.medianLoad.mean())/dfTotalLoadSD.medianLoad.std(ddof=0)
    
    #Remove +/- standard z_score for sweep time
    SD=2
    dfTurnAroundpositionSD.loc[dfTurnAroundpositionSD['sweep1Time_zscore'] >= SD, 'sweep1Time_zscore'] = np.nan
    dfTurnAroundpositionSD.loc[dfTurnAroundpositionSD['sweep1Time_zscore'] <= -SD, 'sweep1Time_zscore'] = np.nan
    dfTurnAroundpositionSD.loc[dfTurnAroundpositionSD['sweep2Time_zscore'] >= SD, 'sweep2Time_zscore'] = np.nan
    dfTurnAroundpositionSD.loc[dfTurnAroundpositionSD['sweep2Time_zscore'] <= -SD, 'sweep2Time_zscore'] = np.nan
    dfTurnAroundpositionSD.loc[dfTurnAroundpositionSD['sweep3Time_zscore'] >= SD, 'sweep3Time_zscore'] = np.nan
    dfTurnAroundpositionSD.loc[dfTurnAroundpositionSD['sweep3Time_zscore'] <= -SD, 'sweep3Time_zscore'] = np.nan
    dfTurnAroundpositionSD.loc[dfTurnAroundpositionSD['sweep4Time_zscore'] >= SD, 'sweep4Time_zscore'] = np.nan
    dfTurnAroundpositionSD.loc[dfTurnAroundpositionSD['sweep4Time_zscore'] <= -SD, 'sweep4Time_zscore'] = np.nan

    #Remove +/-  standard z_score for load
    dfTotalLoadSD.loc[dfTotalLoadSD['mean_zscore'] >= SD, 'meanLoad'] = np.nan
    dfTotalLoadSD.loc[dfTotalLoadSD['mean_zscore'] <= -SD, 'meanLoad'] = np.nan
    dfTotalLoadSD.loc[dfTotalLoadSD['median_zscore'] >= SD, 'medianLoad'] = np.nan
    dfTotalLoadSD.loc[dfTotalLoadSD['median_zscore'] <= -SD, 'medianLoad'] = np.nan
    
    #Remove whole row if a value is NaN, concat metatdata and new values for sweep
    dfTurnAroundpositionSD.loc[dfTurnAroundpositionSD.isnull().any(axis=1), :] = np.nan
    dfTurnAroundpositionSD = dfTurnAroundpositionSD.drop(['Participant', 'Block', 'Fabric', 'Episode Number'], axis=1)
    resultTurnAround = pd.concat([metadataTurnAround, dfTurnAroundpositionSD], axis=1, sort=False)
    
    #Append new values to a new dataframe
    dfTurnAroundFiltered = dfTurnAroundFiltered.append(resultTurnAround, sort = False)
    dfTotalLoadFiltered = dfTotalLoadFiltered.append(dfTotalLoadSD, sort = False)
    

In [266]:
dfTotalLoadAverage = pd.DataFrame(columns=['Participant', 'Fabric', 'meanLoad','medianLoad'])

Create average for toal load

In [267]:

for p in participantNumber: 
    dfP = dfTotalLoadFiltered[dfTotalLoadFiltered['Participant'] == p]
    for f in fabricCode:
        dfF = dfP[dfP['Fabric'] == f]
        print(dfF)
        totalLoadAverage = dfF.dropna()
        totalLoadAverageMean = dfF['meanLoad'].mean()
        totalLoadAverageMedian = dfF['medianLoad'].median()
        dfTotalLoadAverage = dfTotalLoadAverage.append({'Participant': p, 'Fabric': f, 'meanLoad': totalLoadAverageMean, 'medianLoad': totalLoadAverageMedian},ignore_index=True)            

    Participant Block Fabric  Episode Number   meanLoad  medianLoad  \
0           P01    B1      A             1.0  20.573683   21.162972   
1           P01    B1      A             2.0  19.263119   19.347092   
2           P01    B1      A             3.0  17.009524   16.621215   
3           P01    B1      A             4.0  25.596955   22.914790   
4           P01    B1      A             5.0  29.516627   27.430386   
5           P01    B1      A             6.0  41.759879   43.430281   
6           P01    B1      A             7.0  41.460429   41.399459   
7           P01    B1      A             8.0  43.308345   45.785415   
8           P01    B1      A             9.0  53.650533   54.295177   
9           P01    B1      A            10.0  54.498009   55.636152   
10          P01    B1      A            11.0  55.317733   57.139193   
11          P01    B1      A            12.0  53.092333   55.017508   
12          P01    B1      A            13.0  69.005582   69.698864   
13    

809    -0.857961      -0.844583  
    Participant Block Fabric  Episode Number    meanLoad  medianLoad  \
810         P06    B1      A             1.0   77.375876   78.113505   
811         P06    B1      A             2.0   60.359242   57.867785   
812         P06    B1      A             3.0   93.008923   95.759508   
813         P06    B1      A             4.0  110.681558  120.349491   
814         P06    B1      A             5.0  108.962735  108.282220   
815         P06    B1      A             6.0  127.950428  129.081860   
816         P06    B1      A             7.0   98.828244   97.746927   
817         P06    B1      A             8.0         NaN         NaN   
818         P06    B1      A             9.0  109.064236  106.258623   
819         P06    B1      A            10.0   92.182989   93.542818   
820         P06    B1      A            11.0         NaN         NaN   
821         P06    B1      A            12.0   88.110456   91.518288   
822         P06    B1      A  

1601    -0.820506      -0.791172  
     Participant Block Fabric  Episode Number   meanLoad  medianLoad  \
1494         P10    B1      C             1.0        NaN         NaN   
1495         P10    B1      C             2.0  24.590953   25.206482   
1496         P10    B1      C             3.0        NaN         NaN   
1497         P10    B1      C             4.0  19.645750   19.517285   
1498         P10    B1      C             5.0  22.235678   22.922252   
1499         P10    B1      C             6.0  23.685734   24.197853   
1500         P10    B1      C             7.0  25.231507   24.187612   
1501         P10    B1      C             8.0  12.362517   10.966364   
1502         P10    B1      C             9.0  23.430631   22.206370   
1503         P10    B1      C            10.0  22.004978   21.745030   
1504         P10    B1      C            11.0  17.655287   17.107209   
1505         P10    B1      C            12.0  24.768848   26.097348   
1506         P10    B1      C

2555          NaN            NaN  
     Participant Block Fabric  Episode Number    meanLoad  medianLoad  \
2448         P16    B1      B             1.0  101.756209   89.560950   
2449         P16    B1      B             2.0         NaN         NaN   
2450         P16    B1      B             3.0         NaN         NaN   
2451         P16    B1      B             4.0         NaN  114.794861   
2452         P16    B1      B             5.0         NaN         NaN   
2453         P16    B1      B             6.0         NaN         NaN   
2454         P16    B1      B             7.0         NaN         NaN   
2455         P16    B1      B             8.0  116.979693         NaN   
2456         P16    B1      B             9.0   87.520448   78.965168   
2457         P16    B1      B            10.0         NaN         NaN   
2458         P16    B1      B            11.0         NaN         NaN   
2459         P16    B1      B            12.0  113.795045  102.104436   
2460         P16

3383     0.124764       0.187098  
     Participant Block Fabric  Episode Number    meanLoad  medianLoad  \
3276         P21    B1      C             1.0         NaN         NaN   
3277         P21    B1      C             2.0         NaN         NaN   
3278         P21    B1      C             3.0         NaN         NaN   
3279         P21    B1      C             4.0         NaN         NaN   
3280         P21    B1      C             5.0         NaN         NaN   
3281         P21    B1      C             6.0         NaN         NaN   
3282         P21    B1      C             7.0         NaN         NaN   
3283         P21    B1      C             8.0         NaN         NaN   
3284         P21    B1      C             9.0         NaN         NaN   
3285         P21    B1      C            10.0         NaN         NaN   
3286         P21    B1      C            11.0         NaN         NaN   
3287         P21    B1      C            12.0         NaN         NaN   
3288         P21

4193     1.393847       1.535761  
     Participant Block Fabric  Episode Number    meanLoad  medianLoad  \
4086         P26    B1      C             1.0   73.038196   73.935526   
4087         P26    B1      C             2.0   72.859971   70.192903   
4088         P26    B1      C             3.0   74.881850   65.669091   
4089         P26    B1      C             4.0   94.232149   92.644425   
4090         P26    B1      C             5.0         NaN         NaN   
4091         P26    B1      C             6.0         NaN         NaN   
4092         P26    B1      C             7.0         NaN         NaN   
4093         P26    B1      C             8.0         NaN         NaN   
4094         P26    B1      C             9.0         NaN         NaN   
4095         P26    B1      C            10.0         NaN         NaN   
4096         P26    B1      C            11.0         NaN         NaN   
4097         P26    B1      C            12.0         NaN         NaN   
4098         P26

     Participant Block Fabric  Episode Number    meanLoad  medianLoad  \
4860         P31    B1      A             1.0         NaN         NaN   
4861         P31    B1      A             2.0         NaN         NaN   
4862         P31    B1      A             3.0         NaN         NaN   
4863         P31    B1      A             4.0         NaN         NaN   
4864         P31    B1      A             5.0         NaN         NaN   
4865         P31    B1      A             6.0         NaN         NaN   
4866         P31    B1      A             7.0         NaN         NaN   
4867         P31    B1      A             8.0         NaN         NaN   
4868         P31    B1      A             9.0         NaN         NaN   
4869         P31    B1      A            10.0         NaN         NaN   
4870         P31    B1      A            11.0         NaN         NaN   
4871         P31    B1      A            12.0         NaN         NaN   
4872         P31    B1      A            13.0      

Filter dataframe for export

In [268]:
dfTurnAround = dfTurnAroundFiltered[['Participant','Block', 'Fabric', 'Episode Number', 'touchDown', 'sweepStart', 'leftTurn1', 'rightTurn', 'leftTurn2', 'sweepEnd', 'liftOff']]

Export data

In [269]:
dfTurnAround.to_csv(r'C:\Users\jessi\Documents\Touch Data\Turn around times\positionDataEEG.csv', mode='a', index=None)
dfTurnAroundFiltered.to_csv(r'C:\Users\jessi\Documents\Touch Data\Turn around times\positionData.csv', mode='a', index=None)
dfOriginalDataSmooth.to_csv(r'C:\Users\jessi\Documents\Touch Data\Turn around times\OriginalDataSmooth.csv', mode='a', index=None)
dfTotalLoadFiltered.to_csv(r'C:\Users\jessi\Documents\Touch Data\Total Load\totalLoadData.csv', mode='a', index=None)
dfTotalLoadAverage.to_csv(r'C:\Users\jessi\Documents\Touch Data\Total Load\totalLoadDataAverage.csv', mode='a', index=None)